__Basic NLP - Praktek Scraping Tweet__
<br>

oleh:<br>
__Ibnu Pujiono__<br>
__Sigit Sumarsono__<br>

_Ministry of Finance Data Analytics Community (MoF-DAC)_

> Kali ini kita akan mencoba melakukan praktek scraping pada twitter menggunakan bantuan library Tweepy. Dokumentasi [Tweepy](https://docs.tweepy.org/en/stable/).

<details>
    <img src="img/tweepy.jpeg">
</details>

# Proses Scraping pada Twitter

## Installasi Library Tweepy

Perintah yang dilakukan untuk melakukan proses installasi library ```Tweepy```

In [ ]:
!pip install tweepy

## Import Library dan Inisiasi Token

Disini kita harus mengakses dashboard Twitter for Developer yang sudah kita miliki lalu memasukkan ```consumer key``` dan juga ```access token``` untuk mendapatkan otorisasi akses.
<details>
    <img src="img/tweepy-2.jpeg">
</details>

In [ ]:
import tweepy
import pandas as pd
import json



consumer_key = '(isi dengan token masing-masing)'
consumer_secret = '(isi dengan token masing-masing)'
access_token = '(isi dengan token masing-masing)'
access_token_secret = '(isi dengan token masing-masing)'

Proses inisialisai Tweepy menggunakan token yang sudah kita miliki dilakukan dengan fungsi ```AuthHandler```

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Setelah ini berhasil kita baru dapat melakukan proses scraping pada twitter menggunakan akun kita masing-masing. Let's try it :)

In [ ]:
dir(api)

## Scrape Tweet

### Mengambil Tweet dari Akun Tertentu

> Kita dapat mengambil tweet dari timeline akun pribadi kita

In [ ]:
mytimeline = api.home_timeline()
for tweet in mytimeline:
   print (tweet.text)

> Kita juga dapat mengambil tweet dari akun tertentu yang kita kehendaki, contoh disini adalah dari akun ```@kompascom```

In [ ]:
nama = "kompascom"
jumlahtweet = 20

tweets = api.user_timeline(id=nama, count=jumlahtweet)

for tweet in tweets:
   print (tweet.text)

### Melihat Trending Topic

Kita dapat melihat ```Trending Topic``` Twitter dari sebuah lokasi menggunakan woe_id, The Yahoo! Where On Earth ID  atau woe_id merupakan kode lokasi dari Yahoo. Daftar WOE_ID dapat dilihat di [link](https://gist.github.com/edsu/a5f6c1188ec3a27d38634721fb25fffb) ini.

In [ ]:
#Melihat Trending Topic di Indonesia
indonesia_woe = 23424846

trends = api.trends_place(indonesia_woe)
print(trends)

In [ ]:
#Melihat Trending Topic di Newyork
newyorkwoe = 23424977
trends_ny = api.trends_place(newyorkwoe)
#Melihat isi dari Trending Topic
for item in (trends_ny[0]['trends']):
       print (item['name'])

In [ ]:
#Melihat isi dari Trending Topic
for item in (trends[0]['trends']):
       print (item['name'])

Kita juga dapat mengambil ```jumlah tweet``` dari trending topic

In [ ]:
for item in (trends[0]['trends']):
       print(item['tweet_volume'])

Kita juga dapat membuat dataframe yang berisi ```Trending Topic``` dan ```Jumlah Tweet Trending Topic```

In [ ]:
#Membuat List Trending Topic
name = []
for item in (trends[0]['trends']):
       name.append(item['name'])

In [ ]:
#Membuat List Jumlah Tweet
volume = []
for item in (trends[0]['trends']):
       volume.append(item['tweet_volume'])

In [ ]:
#Membuat Dataframe
df_trend = pd.DataFrame({'TrendingTopic': name,
                   'Volume' : volume})

In [ ]:
df_trend.head()

Lalu kita lakukan visualisasi sederhana menggunakan barplot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plot = sns.barplot(data=df_trend[:10], x="TrendingTopic", y="Volume")
plt.title("Jumlah Tweet 10 Trending Topic Teratas")
for item in plot.get_xticklabels():
    item.set_rotation(75)
plt.show()

### Scrape Menggunakan Keyword

Kita dapat juga mencari tweet tertentu dengan menggunakan keyword yang kita tentukan sendiri

In [ ]:
#Mencari Kata Tertentu
search_words = "sumpah pemuda"
date_since = "2021-10-21"
date_until = "2021-10-28"
new_search = search_words + " -filter:retweets"

tweets = tweepy.Cursor(api.search,
        q=new_search,
        lang="id",
        since=date_since,
        until=date_until).items(200)

In [ ]:
import re
import pandas as pd
items = []
for tweet in tweets:
  items.append (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split()))
  hasil_scrape = pd.DataFrame(data=items, columns=['tweet'])

In [ ]:
hasil_scrape.head()

# Analisis Deskriptif Sederhana

## Menghitung Banyak Kata yang muncul

> Kita cleansing data seperti yang sudah kita lakukan sebelumnya

In [ ]:
!pip install nltk
#Proses Pembersihan Data
#Import Library
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
#Bersihin data pake NLTK
#1. Kecilin Huruf
hasil = [item.lower() for item in hasil_scrape.tweet]
#2. Menghilangkan angka, alamat htpp, tanda baca, whitespace
hasil = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", item).split()) for item in hasil]
hasil = [' '.join(re.sub(r"\d+", "", item).split()) for item in hasil]
from nltk.corpus import stopwords
data = []
stopwords = set(stopwords.words('indonesian'))
for item in hasil:
  querywords = item.split()
  resultwords  = [word for word in querywords if word not in stopwords]
  result = ' '.join(resultwords)
  data.append(result)

> Sebelum dimasukkan perlu dilakukan cleansing ulang dengan memasukkan data ke dalam list dan membuat token kata dalam satu list

In [ ]:
import itertools
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
data = [word_tokenize(item) for item in data]
print(data)

In [ ]:
data = list(itertools.chain(*data))
print(data)

> Selanjutnya kita dapat melihat banyaknya kata yang muncul dari tweet yang sudah kita scrape

In [ ]:
fqdist = FreqDist(data)
print(fqdist.most_common(15))

In [ ]:
plt.figure(figsize=(12, 8))
fig,ax = plt.subplots()
fqdist.plot(15,cumulative=False)
plt.show()
fig.savefig('Grafik Kata')

## Membuat Wordcloud

> Sebelum membuat wordcloud kita perlu melakukan installasi library Wordcloud. Dokumentasi dapat dilihat pada [link](http://amueller.github.io/word_cloud/) ini.

In [ ]:
!pip install wordcloud

In [ ]:
#Membuat Wordcloud dari Tweet yang sudah discrape
from wordcloud import WordCloud
import matplotlib.pyplot as plt
data_ok = ' '.join(data)
wordcloud = WordCloud(width=1600, height=1000, max_font_size=200, max_words=30,background_color='white').generate(data_ok)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Wordcloud Keyword Sumpah Pemuda",size=30)
plt.show()